# Producer Function

In [9]:
!pip install confluent_kafka

In [12]:
from confluent_kafka import Producer
import sys
import time
import json


iproducer = None

def kafkaproducer(server='35.236.145.238:9092',topic='test_request',ID='User_ID',query='test'):
    global iproducer
    return_value = 0
    def error_cb(err):
        print('Error: %s' % err)
    props = {
        'bootstrap.servers': server,  # <-- 置換成要連接的Kafka集群
        'error_cb': error_cb                    # 設定接收error訊息的callback函數
    }
    # 步驟2. 產生一個Kafka的Producer的實例
    if iproducer is None:
        iproducer = Producer(props)
    # 步驟3. 指定想要發佈訊息的topic名稱
    topicName = topic
    try:
        # produce(topic, [value], [key], [partition], [on_delivery], [timestamp], [headers])
        
        iproducer.produce(topicName, key=ID, value=query)

        iproducer.flush()

        return_value+=1
    except:
        return_value = 0
    iproducer.flush()
    return return_value


In [ ]:
for i in range(100):
    start = time.time()
    kafkaproducer(server='35.236.145.238:9092',topic='2_resquest',ID='test_ID',query='empty')
    end = time.time()
    print(end-start)
    time.sleep(10)

0.054071903228759766
0.0643308162689209
0.08929443359375
0.04968142509460449
0.04860711097717285
0.05739879608154297
0.03794431686401367
0.04566693305969238
0.05740857124328613
0.06501913070678711
0.05070662498474121
0.03911852836608887


# Consumer Function

In [7]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import sys
consumer = None
def kafkaconsumer(server='35.236.145.238:9092',groupid ='conseumer',topic='test_request1',ID='User_ID'):
    global consumer
    def try_decode_utf8(data):
        if data:
            return data.decode('utf-8')
        else:
            return None

    def my_assign(consumer_instance, partitions):
        for p in partitions:
            p.offset = 0
        consumer_instance.assign(partitions)

    def error_cb(err):
        pass

    props = {
        'bootstrap.servers': server,
        'group.id': groupid,   
        'auto.offset.reset': 'earliest',    
        'session.timeout.ms': 6000,    
        'error_cb': error_cb
    }
    return_answer ={}
    if consumer is None:
        consumer = Consumer(props)
    topicName = topic
    consumer.subscribe([topicName])
    records = []
    while len(records)==0:
        records = consumer.consume(num_messages=1)
        if records is None:
            continue
        for record in records:
            if record is None:
                continue
            if record.error():
                continue
            else:
                msgKey = try_decode_utf8(record.key())
                msgValue = try_decode_utf8(record.value())
                timestamp = record.timestamp()
                
                if str(msgKey) != ID:
                    records = []
                else:
                    t=timestamp
                    #return_answer[msgKey] = (msgValue,t)
                
    return t


In [8]:
start = time.time()
print('counting start')
feedback=kafkaconsumer(server='35.236.145.238:9092',groupid ='t11221',topic='topicName',ID='ID')
print('counting end')
end = time.time()
print('-'*15)
print(end-start)
print(feedback)

counting start
counting end
---------------
6.2071533203125
(1, 1594025263350)


In [ ]:
{'User_ID2': '20200617-test2','prop':}

In [14]:
feedback[1].strftime("%Y-%m-%d %H:%M:%S")

AttributeError: 'str' object has no attribute 'strftime'

In [28]:
import datetime
(datetime.datetime.fromtimestamp(feedback[1] / 1e3)+ datetime.timedelta(hours = 8)).strftime("%Y-%m-%d %H:%M:%S")

'2020-07-06 16:47:43'

In [15]:
print('#'*40)

########################################
